In [ ]:
from pytket import Circuit
from pytket.circuit import OpType
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.allocators import HypergraphPartitioning
from pytket_dqc.circuits import HypergraphCircuit
from pytket_dqc.packing import PacMan

In [ ]:
circuit = Circuit(3)
circuit.add_gate(OpType.CU1,1,[0,1]).H(0).add_gate(OpType.CU1,1,[0,1]).Rz(0.3,0).add_gate(OpType.CU1,1,[0,1]).H(0).add_gate(OpType.CU1,0.5,[0,1]).H(0).add_gate(OpType.CU1,1,[0,1]).H(0).H(0).add_gate(OpType.CU1,1,[0,1]).H(0).add_gate(OpType.CU1,1.5,[0,1])
network = NISQNetwork(
    [[0,1],[0,2],[1,2]],
    {0: [0], 1: [1], 2: [2]}
)
render_circuit_jupyter(circuit)

In [ ]:
hyper_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
placement = distributor.allocate(hyper_circ, network)

In [ ]:
pacman = PacMan(hyper_circ, placement.placement)

In [ ]:
print(pacman.packets_by_qubit)
print(pacman.hopping_packets)